# init

In [9]:
import pandas as pd
import warnings
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

# load data

In [23]:
top_50_areas_df = pd.read_csv('/home/shaghayegh/class/New York project/top50areas.csv')
top_50_areas_df

,Date,LocationID,Week_days,Daily_trips,1_day_lag,2_days_lag,3_days_lag,4_days_lag,5_days_lag,6_days_lag,7_days_lag,8_days_lag,9_days_lag,10_days_lag
0,2023-01-14,13,Saturday,420,572.0,645.0,683.0,633.0,456.0,278.0,417.0,507.0,614.0,493.0
1,2023-01-14,24,Saturday,278,311.0,311.0,300.0,271.0,239.0,225.0,320.0,306.0,288.0,277.0
2,2023-01-14,43,Saturday,1938,1806.0,1696.0,1549.0,1550.0,1376.0,1340.0,1919.0,1994.0,1859.0,1464.0
3,2023-01-14,48,Saturday,3712,2966.0,2865.0,2638.0,2501.0,2039.0,2725.0,3232.0,2787.0,2632.0,2360.0
4,2023-01-14,50,Saturday,648,685.0,632.0,617.0,553.0,464.0,578.0,651.0,628.0,532.0,491.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5345,2023-04-30,246,Sunday,1384,1810.0,1628.0,2119.0,1826.0,1744.0,1394.0,1676.0,2116.0,1568.0,1889.0
5346,2023-04-30,249,Sunday,2490,3530.0,2734.0,2363.0,2081.0,1774.0,1452.0,2419.0,3201.0,2592.0,2296.0
5347,2023-04-30,261,Sunday,440,481.0,551.0,583.0,598.0,497.0,441.0,477.0,611.0,570.0,663.0
5348,2023-04-30,262,Sunday,1214,1445.0,1785.0,1693.0,1735.0,1579.0,1465.0,1221.0,1451.0,1611.0,1795.0


In [24]:
warnings.filterwarnings('ignore')

# prediction 

## train/test split

In [26]:
def train_test_split(dataset_df, split_date):
    train_df = dataset_df[dataset_df['Date'] < split_date]
    test_df = dataset_df[dataset_df['Date'] >= split_date]
    return train_df, test_df

train_df, test_df = train_test_split(top_50_areas_df, '2023-04-01')

In [27]:
train_df

,Date,LocationID,Week_days,Daily_trips,1_day_lag,2_days_lag,3_days_lag,4_days_lag,5_days_lag,6_days_lag,7_days_lag,8_days_lag,9_days_lag,10_days_lag
0,2023-01-14,13,Saturday,420,572.0,645.0,683.0,633.0,456.0,278.0,417.0,507.0,614.0,493.0
1,2023-01-14,24,Saturday,278,311.0,311.0,300.0,271.0,239.0,225.0,320.0,306.0,288.0,277.0
2,2023-01-14,43,Saturday,1938,1806.0,1696.0,1549.0,1550.0,1376.0,1340.0,1919.0,1994.0,1859.0,1464.0
3,2023-01-14,48,Saturday,3712,2966.0,2865.0,2638.0,2501.0,2039.0,2725.0,3232.0,2787.0,2632.0,2360.0
4,2023-01-14,50,Saturday,648,685.0,632.0,617.0,553.0,464.0,578.0,651.0,628.0,532.0,491.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3845,2023-03-31,246,Friday,1526,1798.0,1586.0,1613.0,1306.0,1561.0,1958.0,1561.0,1747.0,1463.0,1340.0
3846,2023-03-31,249,Friday,2786,2210.0,1969.0,1768.0,1382.0,2773.0,3887.0,2652.0,2148.0,1811.0,1649.0
3847,2023-03-31,261,Friday,518,591.0,610.0,472.0,444.0,584.0,583.0,599.0,545.0,549.0,483.0
3848,2023-03-31,262,Friday,1525,1563.0,1445.0,1454.0,1219.0,940.0,1254.0,1499.0,1469.0,1384.0,1404.0


In [28]:
test_df

,Date,LocationID,Week_days,Daily_trips,1_day_lag,2_days_lag,3_days_lag,4_days_lag,5_days_lag,6_days_lag,7_days_lag,8_days_lag,9_days_lag,10_days_lag
3850,2023-04-01,13,Saturday,518,672.0,753.0,760.0,746.0,556.0,402.0,480.0,605.0,740.0,710.0
3851,2023-04-01,24,Saturday,335,318.0,316.0,275.0,238.0,250.0,268.0,269.0,300.0,266.0,272.0
3852,2023-04-01,43,Saturday,1888,1776.0,1686.0,1489.0,1537.0,1435.0,1568.0,1985.0,1714.0,1831.0,1348.0
3853,2023-04-01,48,Saturday,3868,3224.0,3112.0,2934.0,2628.0,2392.0,3034.0,3780.0,3073.0,3114.0,2880.0
3854,2023-04-01,50,Saturday,842,774.0,675.0,668.0,613.0,564.0,712.0,772.0,639.0,645.0,595.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5345,2023-04-30,246,Sunday,1384,1810.0,1628.0,2119.0,1826.0,1744.0,1394.0,1676.0,2116.0,1568.0,1889.0
5346,2023-04-30,249,Sunday,2490,3530.0,2734.0,2363.0,2081.0,1774.0,1452.0,2419.0,3201.0,2592.0,2296.0
5347,2023-04-30,261,Sunday,440,481.0,551.0,583.0,598.0,497.0,441.0,477.0,611.0,570.0,663.0
5348,2023-04-30,262,Sunday,1214,1445.0,1785.0,1693.0,1735.0,1579.0,1465.0,1221.0,1451.0,1611.0,1795.0


In [29]:
FEATURES = ['1_day_lag', '2_days_lag',	'3_days_lag', '4_days_lag',	'5_days_lag', '6_days_lag',	'7_days_lag', '8_days_lag',	'9_days_lag', '10_days_lag']

In [30]:
LABEL = ['Daily_trips']

## methods

### decision tree

#### model

In [31]:
dt = DecisionTreeRegressor(max_depth=16, min_samples_split=30, min_samples_leaf=40)
dt.fit(train_df[FEATURES], train_df[LABEL])

DecisionTreeRegressor(max_depth=16, min_samples_leaf=40, min_samples_split=30)

#### predict

In [32]:
label_pred_test_dt = dt.predict(test_df[FEATURES])

#### save

In [33]:
test_df['Pred_dt'] = label_pred_test_dt
test_df

,Date,LocationID,Week_days,Daily_trips,1_day_lag,2_days_lag,3_days_lag,4_days_lag,5_days_lag,6_days_lag,7_days_lag,8_days_lag,9_days_lag,10_days_lag,Pred_dt
3850,2023-04-01,13,Saturday,518,672.0,753.0,760.0,746.0,556.0,402.0,480.0,605.0,740.0,710.0,463.968750
3851,2023-04-01,24,Saturday,335,318.0,316.0,275.0,238.0,250.0,268.0,269.0,300.0,266.0,272.0,304.833333
3852,2023-04-01,43,Saturday,1888,1776.0,1686.0,1489.0,1537.0,1435.0,1568.0,1985.0,1714.0,1831.0,1348.0,1892.741935
3853,2023-04-01,48,Saturday,3868,3224.0,3112.0,2934.0,2628.0,2392.0,3034.0,3780.0,3073.0,3114.0,2880.0,3651.555556
3854,2023-04-01,50,Saturday,842,774.0,675.0,668.0,613.0,564.0,712.0,772.0,639.0,645.0,595.0,793.204545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5345,2023-04-30,246,Sunday,1384,1810.0,1628.0,2119.0,1826.0,1744.0,1394.0,1676.0,2116.0,1568.0,1889.0,1696.179104
5346,2023-04-30,249,Sunday,2490,3530.0,2734.0,2363.0,2081.0,1774.0,1452.0,2419.0,3201.0,2592.0,2296.0,2543.522727
5347,2023-04-30,261,Sunday,440,481.0,551.0,583.0,598.0,497.0,441.0,477.0,611.0,570.0,663.0,463.968750
5348,2023-04-30,262,Sunday,1214,1445.0,1785.0,1693.0,1735.0,1579.0,1465.0,1221.0,1451.0,1611.0,1795.0,1193.775000


### random forest

In [34]:
rf = RandomForestRegressor(n_estimators=500)
rf.fit(train_df[FEATURES], train_df[LABEL])

RandomForestRegressor(n_estimators=500)

#### predict

In [35]:
label_pred_test_rf = rf.predict(test_df[FEATURES])

#### save

In [36]:
test_df['Pred_rf'] = label_pred_test_rf
test_df

,Date,LocationID,Week_days,Daily_trips,1_day_lag,2_days_lag,3_days_lag,4_days_lag,5_days_lag,6_days_lag,7_days_lag,8_days_lag,9_days_lag,10_days_lag,Pred_dt,Pred_rf
3850,2023-04-01,13,Saturday,518,672.0,753.0,760.0,746.0,556.0,402.0,480.0,605.0,740.0,710.0,463.968750,455.842
3851,2023-04-01,24,Saturday,335,318.0,316.0,275.0,238.0,250.0,268.0,269.0,300.0,266.0,272.0,304.833333,290.414
3852,2023-04-01,43,Saturday,1888,1776.0,1686.0,1489.0,1537.0,1435.0,1568.0,1985.0,1714.0,1831.0,1348.0,1892.741935,1999.510
3853,2023-04-01,48,Saturday,3868,3224.0,3112.0,2934.0,2628.0,2392.0,3034.0,3780.0,3073.0,3114.0,2880.0,3651.555556,3577.452
3854,2023-04-01,50,Saturday,842,774.0,675.0,668.0,613.0,564.0,712.0,772.0,639.0,645.0,595.0,793.204545,745.756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5345,2023-04-30,246,Sunday,1384,1810.0,1628.0,2119.0,1826.0,1744.0,1394.0,1676.0,2116.0,1568.0,1889.0,1696.179104,1672.688
5346,2023-04-30,249,Sunday,2490,3530.0,2734.0,2363.0,2081.0,1774.0,1452.0,2419.0,3201.0,2592.0,2296.0,2543.522727,2535.112
5347,2023-04-30,261,Sunday,440,481.0,551.0,583.0,598.0,497.0,441.0,477.0,611.0,570.0,663.0,463.968750,465.702
5348,2023-04-30,262,Sunday,1214,1445.0,1785.0,1693.0,1735.0,1579.0,1465.0,1221.0,1451.0,1611.0,1795.0,1193.775000,1278.708


### save data set

In [37]:
test_df.to_csv('/home/shaghayegh/class/New York project/test.csv', index=False)